# Introducción a Clusterización

La clusterización (clustering) es una técnica de **aprendizaje no supervisado** que agrupa datos similares en conjuntos llamados clusters. A diferencia del aprendizaje supervisado, no tenemos etiquetas predefinidas.

## ¿Para qué sirve?

- **Segmentación de clientes**: Agrupar clientes con comportamientos similares
- **Detección de anomalías**: Identificar datos que no pertenecen a ningún grupo
- **Compresión de imágenes**: Reducir colores agrupando píxeles similares
- **Organización de documentos**: Agrupar textos por temas

## Algoritmos que veremos

1. **K-Means**: El más popular, agrupa por distancia a centroides
2. **DBSCAN**: Clustering basado en densidad, detecta formas irregulares
3. **Clustering Jerárquico**: Crea una jerarquía de clusters

In [ ]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_blobs, make_moons, load_iris
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score
from scipy.cluster.hierarchy import dendrogram, linkage

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Semilla para reproducibilidad
np.random.seed(42)

---
## Ejemplo 1: K-Means - Datos Sintéticos

K-Means es un algoritmo iterativo que:
1. Inicializa K centroides aleatoriamente
2. Asigna cada punto al centroide más cercano
3. Recalcula los centroides como el promedio de los puntos asignados
4. Repite hasta convergencia

In [ ]:
# Generar datos sintéticos con 4 clusters bien definidos
X_blobs, y_true = make_blobs(n_samples=400, centers=4, n_features=2, 
                              cluster_std=0.8, random_state=42)

# Visualizar los datos originales
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.scatter(X_blobs[:, 0], X_blobs[:, 1], alpha=0.6, s=50)
plt.title('Datos sin etiquetar')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

plt.subplot(1, 2, 2)
plt.scatter(X_blobs[:, 0], X_blobs[:, 1], c=y_true, cmap='viridis', alpha=0.6, s=50)
plt.title('Clusters verdaderos (desconocidos en la práctica)')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar(label='Cluster')

plt.tight_layout()
plt.show()

print(f"Forma de los datos: {X_blobs.shape}")

### Aplicando K-Means

In [ ]:
# Aplicar K-Means con k=4
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
y_kmeans = kmeans.fit_predict(X_blobs)

# Visualizar resultados
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.scatter(X_blobs[:, 0], X_blobs[:, 1], c=y_kmeans, cmap='viridis', alpha=0.6, s=50)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            c='red', marker='X', s=300, edgecolors='black', linewidths=2, label='Centroides')
plt.title('Resultados de K-Means')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.colorbar(label='Cluster')

plt.subplot(1, 2, 2)
# Comparación con clusters verdaderos
plt.scatter(X_blobs[:, 0], X_blobs[:, 1], c=y_true, cmap='viridis', alpha=0.6, s=50)
plt.title('Clusters Verdaderos')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar(label='Cluster')

plt.tight_layout()
plt.show()

# Métricas de evaluación
silhouette = silhouette_score(X_blobs, y_kmeans)
davies_bouldin = davies_bouldin_score(X_blobs, y_kmeans)

print(f"\nMétricas de evaluación:")
print(f"Silhouette Score: {silhouette:.3f} (rango: [-1, 1], mejor: cercano a 1)")
print(f"Davies-Bouldin Index: {davies_bouldin:.3f} (mejor: cercano a 0)")
print(f"Inercia (suma de distancias al cuadrado): {kmeans.inertia_:.2f}")

### ¿Cómo elegir el número de clusters (K)?

El **método del codo** nos ayuda a encontrar el K óptimo observando dónde la inercia deja de disminuir significativamente.

In [ ]:
# Método del codo
inertias = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans_temp.fit(X_blobs)
    inertias.append(kmeans_temp.inertia_)
    silhouette_scores.append(silhouette_score(X_blobs, kmeans_temp.labels_))

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Método del codo
axes[0].plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Número de Clusters (K)', fontsize=12)
axes[0].set_ylabel('Inercia (Within-Cluster Sum of Squares)', fontsize=12)
axes[0].set_title('Método del Codo', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].axvline(x=4, color='red', linestyle='--', label='K óptimo = 4')
axes[0].legend()

# Silhouette Score
axes[1].plot(K_range, silhouette_scores, 'go-', linewidth=2, markersize=8)
axes[1].set_xlabel('Número de Clusters (K)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score vs K', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].axvline(x=4, color='red', linestyle='--', label='K óptimo = 4')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\nObservaciones:")
print("- El 'codo' está en K=4, donde la inercia empieza a disminuir más lentamente")
print("- El Silhouette Score es máximo en K=4, confirmando que es el mejor número de clusters")

---
## Ejemplo 2: K-Means con Dataset Real (Iris)

Usaremos el famoso dataset Iris para segmentar flores según sus características.

In [ ]:
# Cargar dataset Iris
iris = load_iris()
X_iris = iris.data
y_iris_true = iris.target
feature_names = iris.feature_names

# Crear DataFrame para mejor visualización
df_iris = pd.DataFrame(X_iris, columns=feature_names)
df_iris['especie'] = pd.Categorical.from_codes(y_iris_true, iris.target_names)

print("Primeras filas del dataset Iris:")
print(df_iris.head(10))
print(f"\nForma: {df_iris.shape}")
print(f"\nEspecies: {iris.target_names}")

In [ ]:
# Normalizar los datos (importante para K-Means)
scaler = StandardScaler()
X_iris_scaled = scaler.fit_transform(X_iris)

# Aplicar K-Means con k=3 (sabemos que hay 3 especies)
kmeans_iris = KMeans(n_clusters=3, random_state=42, n_init=10)
y_iris_pred = kmeans_iris.fit_predict(X_iris_scaled)

# Visualizar usando las dos primeras características
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Clustering predicho
axes[0].scatter(X_iris[:, 0], X_iris[:, 1], c=y_iris_pred, cmap='viridis', alpha=0.6, s=100)
axes[0].set_xlabel(feature_names[0])
axes[0].set_ylabel(feature_names[1])
axes[0].set_title('K-Means Clustering')

# Especies verdaderas
axes[1].scatter(X_iris[:, 0], X_iris[:, 1], c=y_iris_true, cmap='viridis', alpha=0.6, s=100)
axes[1].set_xlabel(feature_names[0])
axes[1].set_ylabel(feature_names[1])
axes[1].set_title('Especies Verdaderas')

plt.tight_layout()
plt.show()

# Evaluación
silhouette_iris = silhouette_score(X_iris_scaled, y_iris_pred)
print(f"\nSilhouette Score: {silhouette_iris:.3f}")

# Comparar clusters con especies reales
df_comparison = pd.DataFrame({
    'Especie Real': pd.Categorical.from_codes(y_iris_true, iris.target_names),
    'Cluster': y_iris_pred
})
print("\nTabla de contingencia (Especies vs Clusters):")
print(pd.crosstab(df_comparison['Especie Real'], df_comparison['Cluster']))